# Introduction to NVSHMEM

Let's get our first look at [NVSHMEM](https://developer.nvidia.com/nvshmem). NVSHMEM is a [single-program, multiple-data](https://en.wikipedia.org/wiki/SPMD) (SPMD) programming model like MPI, but unlike MPI provides a [partitioned global address space](https://en.wikipedia.org/wiki/Partitioned_global_address_space) (PGAS). In PGAS programming models, you write the program by directly performing memory operations on pointers that refer to memory which reside on either local or on remote memory spaces, and the underlying implementation translates the memory operation into the corresponding local memory operation (if the pointer refers to shared memory data on the local processor) or remote memory operation (if the pointer refers to memory on another processor). Some popular examples of PGAS extensions to programming languages include [Fortran coarrays](https://en.wikipedia.org/wiki/Coarray_Fortran) (which were standardized in Fortran 2008) and [Unified Parallel C](https://en.wikipedia.org/wiki/Unified_Parallel_C). In C++, [UPC++](https://bitbucket.org/berkeleylab/upcxx/wiki/Home) is a popular framework.

## OpenSHMEM

<center><img src="images/OpenSHMEM_committee.png"></center>

We're going to focus on the PGAS programming framework [OpenSHMEM](http://openshmem.org/site/), a standardized model with participation by many HPC vendors, national labs, and universities. In OpenSHMEM, processes have both their standard private memory space as well as a *symmetric* memory space which is eligible to be read from/written to by other processes. Valid memory operations include *put* (write), *get* (read), and *atomic* (read-modify-write) operations as well as collective operations. The private memory space works with standard `malloc()`, while processes opt into the symmetric memory space with `shmem_malloc()`. The individual processes are called PEs or processing elements, which are analogous to MPI ranks, and are similarly zero-indexed.

<center><img src="images/OpenSHMEM_PGAS.png" width="1000"></center>

## NVSHMEM memory model and point-to-point operations

NVSHMEM is an implementation of OpenSHMEM for NVIDIA GPUs, along with a set of NVIDIA-specific extensions to OpenSHMEM. *Symmetric* (device) memory allocations are produced with `nvshmem_malloc()`, while private (device) memory allocations are produced as usual with `cudaMalloc()`.

<center><img src="images/NVSHMEM_PGAS.png" width="1000"></center>

Let's dive right in and see how this works, returning to our example of calculating the value of $\pi$ in parallel. For convenience we'll continue to launch our multiple processes with MPI (this is not strictly required; NVSHMEM can work with [other job launchers that handle interprocess communication](https://docs.nvidia.com/nvshmem/api/using.html#running-nvshmem-programs)). First, let's start with the analogue to [exercises/monte_carlo_mgpu_cuda_mpi.cpp](exercises/monte_carlo_mgpu_cuda_mpi.cpp), where we do the work on each GPU independently and then sum the results at the end. We'll call this first NVSHMEM program [exercises/monte_carlo_nvshmem.cpp](exercises/monte_carlo_nvshmem.cpp).

Let's examine some of the differences. [Compiling](https://docs.nvidia.com/nvshmem/api/using.html#compiling-nvshmem-programs) looks similar to before, but we now need to point to the relevant include and library directories for NVSHMEM (`-I $NVSHMEM_HOME/include -L $NVSHMEM_HOME/lib -lnvshmem`) and also link in the CUDA driver API (`-lcuda`). We also need to add to the code `#include <nvshmem.h>` (for the OpenSHMEM standard-conforming APIs, which look like `nvshmem_*`) and `#include <nvshmemx.h>` (for the NVIDIA extensions, which look like `nvshmemx_*`).

- Initialize NVSHMEM with MPI support

```
    nvshmemx_init_attr_t attr;
    MPI_Comm comm = MPI_COMM_WORLD;
    attr.mpi_comm = &comm;
    nvshmemx_init_attr(NVSHMEMX_INIT_WITH_MPI_COMM, &attr);
```

- Obtain our NVSHMEM processing element ID and the number of PEs

```
    int my_pe = nvshmem_my_pe();
    int n_pes = nvshmem_n_pes();
```

- Allocate device data with `nvshmem_malloc()` instead of `cudaMalloc()`

```
    int* d_hits = (int*) nvshmem_malloc(sizeof(int));
```

- Initialize data on the device with `nvshmem_int_p()` instead of `cudaMemcpy()` or `cudaMemset()`

```
    nvshmem_int_p(d_hits, 0, my_pe);
```

- Synchronize across PEs to ensure all kernels on all devices have completed (this was done implicitly for us earlier by `MPI_Reduce()`)

```
    nvshmem_barrier_all();
```

- Calculate the result on PE 0, using `nvshmem_int_g` to obtain the result from each PE

```
    if (my_pe == 0) {
        int total_hits = 0;
        for (int i = 0; i < n_pes; ++i) {
            total_hits += nvshmem_int_g(d_hits, i);
        }
        ...
    }
```

In the above, [nvshmem_malloc](https://docs.nvidia.com/nvshmem/archives/nvshmem-101/api/docs/gen/api/memory.html) is an implementation of [shmem_malloc](https://www.open-mpi.org/doc/v3.1/man3/shmem_malloc.3.php), which allocates memory from the *symmetric heap*. We say that the returned address is a *symmetric address*.

<center><img src="images/NVSHMEM_put.png" width="500" height="500">&nbsp&nbsp&nbsp&nbsp<img src="images/NVSHMEM_get.png" width="500" height="500"></center>

[nvshmem_int_p](https://docs.nvidia.com/nvshmem/api/gen/api/rma.html#nvshmem-p) and [nvshmem_int_g](https://docs.nvidia.com/nvshmem/api/gen/api/rma.html#nvshmem-g) are, respectively, scalar *put* and *get* operations who take as arguments the symmetric address and the target PE (the *put* API also includes, of course, the value to set). Notice that the same API works on both local and remote data. Notice also that we're calling the API on the host but accessing data that devices on the device. Many NVSHMEM APIs have both host and device implementations and just do the right thing depending on where they're called from. Finally, pay attention to the one-sided nature of these operations. In the last code snippet, PE 0 is performing a `get` operation on data from other PEs, but the other PEs don't participate in this operation! This makes it much more natural and flexible to write code (compared to traditional two-sided MPI operations that are most programmers' first introduction to distributed memory programming, where you have to be cognizant of writing symmetric code both the sender and the receiver). The image above depicts what the put and get operations (respectively) look like when performed on the GPU.

As usual, there are simple `FIXME` tasks in [exercises/monte_carlo_nvshmem.cpp](exercises/monte_carlo_nvshmem.cpp), and you can consult [the solution](solutions/monte_carlo_nvshmem.cpp) if you need help.

In [1]:
NUM_DEVICES = !nvidia-smi -L | wc -l
NUM_DEVICES = int(NUM_DEVICES[0])

In [3]:
!nvcc -ccbin=mpicxx -x cu -arch=sm_70 -I $NVSHMEM_HOME/include -L $NVSHMEM_HOME/lib -lnvshmem -lcuda -o monte_carlo_nvshmem solutions/monte_carlo_nvshmem.cpp
!mpirun -np $NUM_DEVICES ./monte_carlo_nvshmem

/root/nvshmem/include/nvshmem_common.cuh(434): warning: extern declaration of the entity nvshmemi_mype_d is treated as a static definition

/root/nvshmem/include/nvshmem_common.cuh(435): warning: extern declaration of the entity nvshmemi_npes_d is treated as a static definition

/root/nvshmem/include/nvshmem_common.cuh(436): warning: extern declaration of the entity nvshmemi_node_mype_d is treated as a static definition

/root/nvshmem/include/nvshmem_common.cuh(437): warning: extern declaration of the entity nvshmemi_node_npes_d is treated as a static definition

/root/nvshmem/include/nvshmem_common.cuh(438): warning: extern declaration of the entity nvshmemi_p2p_attrib_native_atomic_support_d is treated as a static definition

/root/nvshmem/include/nvshmem_common.cuh(439): warning: extern declaration of the entity nvshmemi_proxy_d is treated as a static definition

/root/nvshmem/include/nvshmem_common.cuh(440): warning: extern declaration of the entity nvshmemi_atomics_sync_d is treat

Congratulations! You've successfully run your first NVSHMEM program.

## Collectives

In the previous exercise we used a point-to-point memory operation, (scalar) get, to perform the final reduction on the number of hits. When we get to large scale this is not a desirable way to perform the reduction. Instead we can perform this directly as a *collective* operation where multiple PEs participate. A group of PEs which can participate in a collective operation is called a *team* (analogous to the MPI concept of a communicator). NVSHMEM provides by default the team `NVSHMEM_TEAM_WORLD` which includes all PEs, and [other teams can be created](https://docs.nvidia.com/nvshmem/api/gen/api/teams.html) as subsets of that world team. Actually, we haev already encountered a collective operation, `nvshmem_barrier_all()`, which implicitly operates on all PEs.

<center><img src="images/NVSHMEM_reduction.png"></center>

We are performing a sum reduction of integers, so we want to use the API [nvshmem_int_sum_reduce(team, dest, source, nreduce)](https://docs.nvidia.com/nvshmem/api/gen/api/collectives.html#sum). Both the source and the destination must be symmetric addresses (e.g. allocated with `nvshmem_malloc`), and all PEs in the team must participate in the operation (unlike the one-sided put/get operations we used before). As a result, the code now looks like

```
    // Accumulate the results across all PEs
    int* d_total_hits = (int*) nvshmem_malloc(sizeof(int));
    nvshmem_int_sum_reduce(NVSHMEM_TEAM_WORLD, d_total_hits, d_hits, 1);
```

Try this out in [exercises/monte_carlo_nvshmem_reduce.cpp](exercises/monte_carlo_nvshmem_reduce.cpp). As usual, consult the [solution](solutions/monte_carlo_nvshmem_reduce.cpp) if you get stuck.

In [5]:
!nvcc -ccbin=mpicxx -x cu -arch=sm_70 -rdc=true -I $NVSHMEM_HOME/include -L $NVSHMEM_HOME/lib -lnvshmem -lcuda -o monte_carlo_nvshmem_reduce solutions/monte_carlo_nvshmem_reduce.cpp
!mpirun -np $NUM_DEVICES ./monte_carlo_nvshmem_reduce

src/comm/transports/ibrc/ibrc.cpp:1445: NULL value get_device_list failed 
src/comm/transports/ibrc/ibrc.cpp:1445: NULL value get_device_list failed 
src/comm/transports/ibrc/ibrc.cpp:1445: NULL value get_device_list failed 
src/comm/transports/ibrc/ibrc.cpp:1445: NULL value get_device_list failed 
Estimated value of pi = 3.14072
Error = 0.000277734


## Summary

We've now encountered NVSHMEM for the first time, and learned how to perform both point-to-point and collective operations. However, until now we've only used it like we would have used traditional MPI -- from the host. In the next section we'll start writing NVSHMEM operations in device code.